In [ ]:
!pip install sentencepiece

In [3]:
import torch
from fairseq.models.lstm import LSTMModel
from fairseq.data.encoders import register_bpe

class SentencePieceBPE(object):
    def __init__(self, args):
        import sentencepiece as spm
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(args.spm_model_path)

    def encode(self, x: str) -> str:
        return self.sp.EncodeAsPieces(x)

    def decode(self, x: str) -> str:
        return self.sp.DecodePieces(x)


# Register SentencePieceBPE as BPE for Fairseq
register_bpe("sentencepiece", SentencePieceBPE)

# Load the Fairseq trained LSTM model
model_path = "lstm_model"
model = LSTMModel.from_pretrained(
    model_path,
    checkpoint_file="lstm.pt",
    data_name_or_path="en-ar-lstm",
    source_lang="ar",  # Specify the source language code
    target_lang="en",  # Specify the target language code
    bpe="sentencepiece",  # Use SentencePiece BPE
    sentencepiece_vocab="ar-en.32kspm.vocab",  # Path to SentencePiece vocabulary
    spm_model_path="ar-en.32kspm.model",  # Path to SentencePiece model
    sentencepiece_model="ar-en.32kspm.model",  # Path to SentencePiece model
)

# Set the model to evaluation mode
model.eval()

# Translate interactively
while True:
    # Take user input
    input_sentence = input("Enter a sentence in English (or 'q' to quit): ")

    if input_sentence.lower() == "q":
        break

    # Tokenize the input sentence
    tokens = model.encode(input_sentence)

    # Convert the tokens to PyTorch tensor
    input_tensor = torch.LongTensor(tokens).unsqueeze(0)  # Add batch dimension

    # Generate translation using the model
    with torch.no_grad():
        translation = model.generate(input_tensor, beam=5)

    # Get the translated sentence without special tokens
    translation_sentence = model.decode(translation[0][0]["tokens"])

    print("Translated Sentence:", translation_sentence)


2023-06-25 01:04:43 | INFO | fairseq.file_utils | loading archive file lstm_model
2023-06-25 01:04:43 | INFO | fairseq.file_utils | loading archive file en-ar-lstm
2023-06-25 01:04:44 | INFO | fairseq.tasks.translation | [ar] dictionary: 32488 types
2023-06-25 01:04:44 | INFO | fairseq.tasks.translation | [en] dictionary: 19704 types
2023-06-25 01:04:44 | INFO | fairseq.models.fairseq_model | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'multilangual lstm en to ar', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 

Enter a sentence in English (or 'q' to quit):  narjes


Translated Sentence: Constantjes


Enter a sentence in English (or 'q' to quit):  السلام عليكن


Translated Sentence: Peace for you


...

Enter a sentence in English (or 'q' to quit):  لا


Translated Sentence: No. No.


Enter a sentence in English (or 'q' to quit):  أحمد ذهب إلى المدرسة


Translated Sentence: Ahmad went to school


KeyboardInterrupt: Interrupted by user